In [1]:
import mysql.connector
import csv
import pandas as pd
import os
import tempfile
import re
import time
from random import randint, uniform

In [2]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

In [3]:
con = mysql.connector.connect(user='andr',
                              password='rstq!2Ro',
                              host='127.0.0.1',
                              database='cat_db',
                              auth_plugin='mysql_native_password'
                             )

In [4]:
curA = con.cursor(dictionary=True, buffered=True)
curB = con.cursor(dictionary=True, buffered=True)
curC = con.cursor(dictionary=True, buffered=True)


## Student texts
Now let's take some example student texts and extract tagsets and POS tagging using conllu API.

In [5]:
from conllu import parse, parse_tree

In [6]:
def parser(filename):
    """
    Yields a sentence from conllu tree with its tags

    """
    """
    >>> for i in parser('/content/gdrive/My Drive/Новые conll по доменам/NewVers/CleanedPsyEdu.conllu'):
      print(i)   
    TokenList<Музыка, звучит, отовсюду, независимо, от, нашего, желания, или, нежелания, слушать, ее, .>
    """
    with open(filename, 'r', encoding='utf-8') as f:
        data = f.read()
    tree = parse(data)
    for token in tree:
        yield token

In [7]:
def get_words(tree):
    """
    tree - generator of sentences (TokenLists) from conllu tree

    words, list is a list of all tokens we need from the tree
    size, int is a number of all words in the domain
    """
    words = []
    for sentence in tree:
        for token in sentence:
#             print(token)
            if token['form'] != '_' and token['upostag'] != '_' and token['upostag']!='NONLEX' and token['form'] not in r'[]\/':
                for wordform in token['form'].split(): # .lower()
                    words.append((wordform, token['lemma'], token['feats'], token['upostag']))
    size = len(words)
    return words, size

#### Добавляем словарь nplus

In [8]:
with open('nplus.csv', mode='r', encoding='utf-8') as infile:
    reader = csv.reader(infile)
    nplus = {(rows[0], rows[1]):rows[2] for rows in reader}

#### Папки с студентическими текстами

In [9]:
stud_dir = r'C:\Users\Andrea\Desktop\stud_textVSscie_text\Student_texts_for_experiments\stud_txt'
low_lvl = os.path.join(stud_dir, 'Low Level')
reg_lvl = os.path.join(stud_dir, 'Regular Level')
low_prsd = os.path.join(stud_dir, 'conllu', 'Low_Level_Parsed')
reg_prsd = os.path.join(stud_dir, 'conllu', 'Regular_Level_Parsed')
ma_theses=r'C:\Users\Andrea\Desktop\stud_textVSscie_text\Student_texts_for_experiments\Fin_MA_theses_parsed'

In [10]:
def tagset_lemma(words):
    print('tagset being created...')
    with open('tagset.csv', 'w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['token', 'lemma', 'tagset', 'POS'])

        for word in words:
            if word[2]:
                tag_lst = []
                for tag in list(word[2].items()):
                    tag = '{}={}|'.format(tag[0], tag[1])
                    tag_lst.append(tag)

                tag_str = ''.join([str(elem) for elem in tag_lst])
                tag_str = tag_str[:-1]

                writer.writerow([word[0], word[1], tag_str, word[3]])
            # print(tag_str)
            else:
                tag_str = 'None'
                writer.writerow([word[0], word[1], tag_str, word[3]])
                
    df = pd.read_csv('tagset.csv')
    word_list = df.values.tolist()
    return word_list

In [11]:
curX = con.cursor(dictionary=True, buffered=True)

In [12]:
curX.execute("""SELECT unigram, lemm, morph, pos FROM
                (SELECT unigram, morph, lemma FROM unigrams) AS a JOIN
                (SELECT id_lemmas, id_pos, lemma AS lemm FROM lemmas) AS b ON lemma = id_lemmas JOIN pos ON b.id_pos = pos.id_pos
                WHERE unigram="интерактивном" &&
                lemm="интерактивный" &&
                morph="Case=Loc|Degree=Pos|Gender=Masc|Number=Sing" && 
                pos="AD";""")

In [13]:
rowsX = curX.fetchall()

In [14]:
if rowsX:
    print(True)

In [15]:
def morph_error_catcher(words):
    mistakes = {}
    corrects = {}
    curX = con.cursor(dictionary=True, buffered=True)
    for i, word in enumerate(words):
        time.sleep(uniform(0.2, 0.6))
        curX.execute("""SELECT unigram, lemm, morph, pos FROM
                (SELECT unigram, morph, lemma FROM unigrams) AS a JOIN
                (SELECT id_lemmas, id_pos, lemma AS lemm FROM lemmas) AS b ON lemma = id_lemmas JOIN pos ON b.id_pos = pos.id_pos
                WHERE unigram="{}" &&
                lemm="{}" &&
                morph="{}" && 
                pos="{}";""".format(word[0], word[1], word[2], word[3]))  # word[2],
        rowsX = curX.fetchall()
        if not rowsX:
            mistakes[i] = [word[0], word[1], word[2], word[3]] # word[2],
        else:
            corrects[i] = [word[0], word[1], word[2], word[3]] # word[2],
    print(mistakes, "\n\n", corrects, "\n\n")
    return mistakes, corrects

In [16]:
from string import punctuation
punctuation += '«»—…“”–•'
punctuation = set(punctuation)
from nltk.corpus import stopwords
stops = stopwords.words('russian')

In [17]:
corr_dir = r'C:\Users\Andrea\Desktop\corrected_files'

In [18]:
reg = re.compile(".+[a-z]+.+")

In [19]:
numbers = re.compile("[0-9]")
# latins = re.compile("[a-zA-Z]")
latins = re.compile("([a-zA-Z]+\W+)|(\W+[a-zA-Z]+)|(\W+[a-zA-Z]\W+)|([a-zA-Z]+)")
cyrillic = re.compile("([а-яА-ЯёЁ]+\W+)|(\W+[а-яА-ЯёЁ]+)|(\W+[а-яА-ЯёЁ]\W+)")
initial = re.compile("[а-яА-ЯёЁ]\.")

In [20]:
def correctionX(filepath, corrected_files_directory):
    # the input must be a conllu format file
    
    filename=os.path.basename(os.path.normpath(filepath))
    # tagset creation
    tree = parser(filepath)
    words, size = get_words(tree)
    del tree   
    
    # correction process
    tot=0
    mists=0
    tagset = tagset_lemma(words)
#     print(tagset)
    correction = []
    mistakes = morph_error_catcher(tagset)[0]
#     print(mistakes)
    # for idx in mistakes:
    for i, word in enumerate(tagset):
#         print(word)
        if i in mistakes and word[0].lower() not in punctuation and word[0].lower() not in stops and \
        (word[0].lower(), word[1].lower()) not in nplus and not numbers.match(word[0].lower()) and \
        not latins.search(word[0].lower()) and not cyrillic.search(word[0].lower()) and word[3] != 'PROPN':
            correction.append('++'+word[0]+'++')
            mists+=1
            tot+=1
        else:
            correction.append(word[0])
            tot+=1
    correction = ' '.join(correction)
    os.makedirs(corrected_files_directory, exist_ok=True)
    with open(os.path.join(corrected_files_directory, filename[:-6]+'txt'), 'w', encoding='utf-8') as fw:
        fw.write(correction)
    return print(correction, "\n\nCorrected words: %s" %mists, "\nMistake frequency: %s" %(mists/tot))

### Пример 1

In [22]:
correctionX(os.path.join(low_prsd, 'prs_EC12_B1_2421.conllu'), corr_dir)

tagset being created...
{0: ['<B1', '<B1', 'None', 'NUM'], 1: ['2421', '2421', 'None', 'NUM'], 2: ['>', '>', 'None', 'SYM'], 3: ['<Russian', '<Russiaн-лат', 'Foreign=Yes', 'PROPN'], 4: ['“н”=N', '“н”=N', 'Foreign=Yes', 'PROPN'], 5: ['>', '>', 'None', 'SYM'], 6: ['<о', '<о', 'Degree=Pos', 'ADV'], 8: ['=', '=', 'None', 'SYM'], 9: ['у', 'у', 'None', 'ADP'], 11: [',', ',', 'None', 'PUNCT'], 12: ['нето', 'нето', 'Case=Nom', 'PRON'], 13: ['=', '=', 'None', 'SYM'], 15: ['>', '>', 'None', 'SYM'], 20: ['о', 'о', 'None', 'ADP'], 22: ['.', '.', 'None', 'PUNCT'], 23: ['По', 'по', 'None', 'ADP'], 25: [',', ',', 'None', 'PUNCT'], 28: ['из', 'из', 'None', 'ADP'], 32: ['в', 'в', 'None', 'ADP'], 34: ['.', '.', 'None', 'PUNCT'], 35: ['Нада', 'надо', 'Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing', 'NOUN'], 38: ['с', 'с', 'None', 'ADP'], 41: ['рассчитаивайте', 'рассчитаивать', 'Aspect=Imp|Mood=Imp|Number=Plur|Person=2|VerbForm=Fin|Voice=Act', 'VERB'], 42: [',', ',', 'None', 'PUNCT'], 43: ['ну', 'ну', 'Non

### Пример 2

In [23]:
correctionX(os.path.join(low_prsd, 'prs_EC12-B1-0404.conllu'), corr_dir)

tagset being created...
{4: ['?', '?', 'None', 'PUNCT'], 8: ['?', '?', 'None', 'PUNCT'], 12: ['?', '?', 'None', 'PUNCT'], 13: ['Кажде', 'Кажде', 'None', 'ADP'], 19: ['.', '.', 'None', 'PUNCT'], 21: ['луди', 'луди', 'None', 'ADP'], 28: ['видет', 'видеть', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'VERB'], 29: ['кажде', 'кажде', 'None', 'ADP'], 31: ['и', 'и', 'None', 'CCONJ'], 33: ['снеме', 'снема', 'Animacy=Anim|Case=Dat|Gender=Fem|Number=Sing', 'NOUN'], 34: ['.', '.', 'None', 'PUNCT'], 35: ['Но', 'но', 'None', 'CCONJ'], 36: ['на', 'на', 'None', 'ADP'], 38: ['старане', 'старана', 'Animacy=Anim|Case=Nom|Gender=Masc|Number=Plur', 'NOUN'], 40: ['то', 'то', 'None', 'SCONJ'], 44: ['как', 'как', 'None', 'SCONJ'], 46: ['и', 'и', 'None', 'CCONJ'], 47: ['не', 'не', 'None', 'PART'], 48: ['знаит', 'знаить', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'VERB'], 51: ['кажде', 'кажде', 'None', 'ADP'], 53: ['.', '.', 'None', 'PUNCT']

### Пример 3 - средний уровень

In [24]:
correctionX(os.path.join(reg_prsd, 'prs_EC12_B1_4930.conllu'), corr_dir)

tagset being created...
{0: ['<B1', '<B1', 'None', 'NUM'], 1: ['4930', '4930', 'None', 'NUM'], 2: ['>', '>', 'None', 'SYM'], 3: ['<issues', '<issues', 'Foreign=Yes', 'X'], 5: ['linking', 'linking', 'Foreign=Yes', 'X'], 7: ['>', '>', 'None', 'SYM'], 11: ['?', '?', 'None', 'PUNCT'], 12: ['В', 'в', 'None', 'ADP'], 13: ['нашой', 'нашой', 'Case=Loc|Degree=Pos|Gender=Fem|Number=Sing', 'ADJ'], 19: ['.', '.', 'None', 'PUNCT'], 21: ['домает', 'домать', 'Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin|Voice=Act', 'VERB'], 22: ['по', 'по', 'None', 'ADP'], 24: [',', ',', 'None', 'PUNCT'], 25: ['из', 'из', 'None', 'ADP'], 27: ['стациях', 'стация', 'Animacy=Inan|Case=Loc|Gender=Fem|Number=Plur', 'NOUN'], 28: [',', ',', 'None', 'PUNCT'], 29: ['с', 'с', 'None', 'ADP'], 32: ['.', '.', 'None', 'PUNCT'], 33: ['Но', 'но', 'None', 'CCONJ'], 38: ['с', 'с', 'None', 'ADP'], 40: ['и', 'и', 'None', 'CCONJ'], 41: ['которих', 'которий', 'Animacy=Anim|Case=Acc|Degree=Pos|Number=Plur', 'ADJ'], 43: 

### Пример 4 - средний уровень

In [25]:
correctionX(os.path.join(reg_prsd, 'prs_EC12-B1-0732.conllu'), corr_dir)

tagset being created...
{3: ['?', '?', 'None', 'PUNCT'], 5: ['не', 'не', 'None', 'PART'], 8: ['.', '.', 'None', 'PUNCT'], 10: [',', ',', 'None', 'PUNCT'], 13: ['скем', 'скть', 'Aspect=Imp|Mood=Ind|Number=Plur|Person=1|Tense=Pres|VerbForm=Fin|Voice=Act', 'VERB'], 17: [',', ',', 'None', 'PUNCT'], 18: ['с', 'с', 'None', 'ADP'], 21: ['висилится', 'висилиться', 'Aspect=Perf|Mood=Ind|Number=Sing|Person=3|Tense=Fut|VerbForm=Fin|Voice=Mid', 'VERB'], 22: [',', ',', 'None', 'PUNCT'], 23: ['с', 'с', 'None', 'ADP'], 27: ['о', 'о', 'None', 'ADP'], 29: ['и', 'и', 'None', 'CCONJ'], 30: ['о', 'о', 'None', 'ADP'], 32: ['.', '.', 'None', 'PUNCT'], 35: ['для', 'для', 'None', 'ADP'], 37: ['когда', 'когда', 'None', 'SCONJ'], 40: ['и', 'и', 'None', 'CCONJ'], 41: ['даже', 'даже', 'None', 'PART'], 42: ['когда', 'когда', 'None', 'SCONJ'], 43: ['не', 'не', 'None', 'PART'], 45: ['ну', 'ну', 'None', 'PART'], 50: ['для', 'для', 'None', 'ADP'], 52: ['просто', 'просто', 'None', 'PART'], 54: ['-', '-', 'None', 'PUNCT

### MA Thesis

In [28]:
correctionX(os.path.join(ma_theses, 'Text1.conllu'), corr_dir)

tagset being created...
{0: ['4', '4', 'None', 'NUM'], 1: ['1', '1', 'None', 'NUM'], 2: ['.', '.', 'None', 'PUNCT'], 10: ['на', 'на', 'None', 'ADP'], 12: [',', ',', 'None', 'PUNCT'], 17: ['о', 'о', 'None', 'ADP'], 19: ['.', '.', 'None', 'PUNCT'], 21: ['и', 'и', 'None', 'CCONJ'], 23: ['от', 'от', 'None', 'ADP'], 28: ['не', 'не', 'None', 'PART'], 29: ['только', 'только', 'None', 'PART'], 30: ['в', 'в', 'None', 'ADP'], 32: [',', ',', 'None', 'PUNCT'], 33: ['но', 'но', 'None', 'CCONJ'], 34: ['и', 'и', 'None', 'PART'], 36: ['в', 'в', 'None', 'ADP'], 40: ['.', '.', 'None', 'PUNCT'], 42: [',', ',', 'None', 'PUNCT'], 45: [',', ',', 'None', 'PUNCT'], 47: ['только', 'только', 'None', 'PART'], 48: ['одной', 'один', 'Case=Gen|Gender=Fem', 'NUM'], 51: ['и', 'и', 'None', 'CCONJ'], 52: ['затушевывание', 'затушевывание', 'Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing', 'NOUN'], 54: [',', ',', 'None', 'PUNCT'], 58: ['или', 'или', 'None', 'CCONJ'], 61: ['при', 'при', 'None', 'ADP'], 64: ['—', '—', 'None

4 1 . ВВЕДЕНИЕ Передача политической речи оказывает сильное влияние на представление , которое читатель составляет себе о политике . Неточные и оторванные от контекста цитаты являются привычными не только в газетах , но и повсюду в средствах массовой информации . Например , подбор фактов , акцентирование только одной стороны явления и ++затушевывание++ другой , употребление оценочных метафор или эвфемистических перифразов при передаче новостей — все это сильно влияет на восприятие . Целью данной работы является определить способы , используемые журналистами при ++реферировании++ и цитировании политической речи : как называется оратор , какие глаголы и наречия употребляются и так далее . На этой основе можно создать представление о том , как выражается субъективный элемент , то есть точка зрения журналиста . Когда журналист пишет статью , ему надо выбрать подходящий стиль и жанр . После выбора того или иного жанра ++пишущий++ может бессознательно привнести те или иные стилевые элементы 

In [21]:
correctionX(os.path.join(ma_theses, 'Text2.conllu'), corr_dir)

tagset being created...
{3: [',', ',', 'None', 'PUNCT'], 4: ['и', 'и', 'None', 'CCONJ'], 8: [',', ',', 'None', 'PUNCT'], 9: ['и', 'и', 'None', 'CCONJ'], 13: ['и', 'и', 'None', 'CCONJ'], 17: [',', ',', 'None', 'PUNCT'], 18: ['и', 'и', 'None', 'CCONJ'], 22: ['в', 'в', 'None', 'ADP'], 26: ['.', '.', 'None', 'PUNCT'], 27: ['В', 'в', 'None', 'ADP'], 28: ['Европейском', 'Европейский', 'None', 'PROPN'], 31: ['-', '-', 'None', 'PUNCT'], 36: [',', ',', 'None', 'PUNCT'], 44: ['-', '-', 'None', 'PUNCT'], 46: [',', ',', 'None', 'PUNCT'], 52: ['.', '.', 'None', 'PUNCT'], 53: ['В', 'в', 'None', 'ADP'], 58: ['и', 'и', 'None', 'CCONJ'], 61: ['и', 'и', 'None', 'CCONJ'], 63: [',', ',', 'None', 'PUNCT'], 64: ['но', 'но', 'None', 'CCONJ'], 66: ['и', 'и', 'None', 'CCONJ'], 70: ['в', 'в', 'None', 'ADP'], 73: ['.', '.', 'None', 'PUNCT'], 74: ['На', 'на', 'None', 'ADP'], 75: ['обуче-ние', 'обуче-ние', 'Animacy=Inan|Case=Acc|Gender=Neut|Number=Sing', 'NOUN'], 79: ['многие', 'много', 'None', 'NUM'], 81: [',', '

-------------------------------------------------

In [52]:
from ufal.udpipe import Model, Pipeline, ProcessingError
import os
import re
from nltk import sent_tokenize
from nltk.tokenize import word_tokenize